## 3.9 dict和set的背后

### 3.9.3 dict的实现及其导致的结果
下面的内容会讨论使用散列表给 dict 带来的优势和限制都有哪些。

#### 01. 键必须是可散列的

一个可散列的对象必须满足以下要求。  
+ (1) 支持 hash() 函数，并且通过 \_\_hash\_\_() 方法所得到的散列值是不变的。  
+ (2) 支持通过 \_\_eq\_\_() 方法来检测相等性。  
+ (3) 若 a == b 为真，则 hash(a) == hash(b) 也为真。  

所有由用户自定义的对象默认都是可散列的，因为它们的散列值由id()来获取，而且它们都是不相等的。  

#### 02. 字典在内存上的开销巨大

由于字典使用了散列表，而散列表又必须是稀疏的，这导致它在空间上的效率低下。  
举例而言，如果你需要存放数量巨大的记录，那么放在由元组或是具名元组构成的列表中会是比较好的选择; 最好不要根据JSON的风格，用由字典组成的列表来存放这些记录。用元组取代字典就能节省空间的原因有两个:其一是避免了散列表所耗费的空间，其二是无需把记录中字段的名字在每个元素里都存一遍。  

在用户自定义的类型中，\_\_slots\_\_ 属性可以改变实例属性的存储方式，由 dict 变成 tuple。

#### 03. 键查询很快  

dict 的实现是典型的空间换时间: 字典类型有着巨大的内存开销，但它们提供了无视数据量大小的快速访问——只要字典能被装在内存里。

#### 04. 键的次序取决于添加顺序

当往 dict 里添加新键而又发生散列冲突的时候，新键可能会被安排存放到另一个位置。  
于是下面这种情况就会发生: 
+ 由 dict([key1, value1), (key2, value2)] 和 dict([key2, value2], [key1, value1]) 得到的两个字典， 在进行比较的时候，它们是相等的; 但是如果在 key1 和 key2 被添加到字典里的过程中有冲突发生的话，这两个键出现在字典里的顺序是不一样的。  

以下这个示例用同样的数据创建了 3 个字典，唯一的区别就是数据出现的顺序不一样。可以看到，虽然键的次序是乱的，这 3 个字典仍然被视作相等的。

In [9]:
# 世界人口数量前10位国家的电话区号 
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
    ]

d1 = dict(DIAL_CODES)  # 创建 d1 的时候，数据元组的顺序是按照国家的人口排名来决定的。
print('d1:', d1.keys())

d2 = dict(sorted(DIAL_CODES))  # 创建 d2 的时候，数据元组的顺序是按照国家的电话区号来决定的。
print('d2:', d2.keys())

d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))  # 创建 d3 的时候，数据元组的顺序是按照国家名字的英文拼写来决定的。
print('d3:', d3.keys())

d1: dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])
d2: dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])
d3: dict_keys([880, 55, 86, 91, 62, 81, 234, 92, 7, 1])


In [10]:
assert d1 == d2 and d2 == d3
# 不报错，说明 assert 后表达式为真

#### 05. 往字典里添加新键可能会改变已有键的顺序

无论何时往字典里添加新的键，Python 解释器都可能做出为字典扩容的决定。扩容导致的结果就是要新建一个更大的散列表，并把字典里已有的元素添加到新表里。  
这个过程中可能会发生新的散列冲突，导致新散列表中键的次序变化。要注意的是，上面提到的这些变化是否会发生以及如何发生，都依赖于字典背后的具体实现，因此你不能很自信地说自己知道背后发生了什么。  
如果你在迭代一个字典的所有键的过程中同时对字典进行修改，那么这个循环很有可能会跳过一些键——甚至是跳过那些字典中已经有的键。  

由此可知，不要对字典同时进行迭代和修改。如果想扫描并修改一个字典，最好分成两步来进行: 首先对字典迭代，以得出需要添加的内容，把这些内容放在一个新字典里;迭代结束之后再对原有字典进行更新。

#### 总结

+ 集合里的元素必须是可散列的。
+ 集合很消耗内存。
+ 可以很高效地判断元素是否存在于某个集合。
+ 元素的次序取决于被添加到集合里的次序。
+ 往集合里添加元素，可能会改变集合里已有元素的次序。